In [248]:
import jieba as jieba
import itertools
import codecs
import nltk
import numpy as np
import re
from datetime import datetime

In [220]:
unknow_token = "UNKNOWN"
sentence_start_token = "START"
sentence_end_token = "END"
sentences = []
with codecs.open('data/殺手_迴光返照的命運.txt', 'r', 'utf-8') as f:
    plainSentence = f.readlines()
    for sentence in plainSentence[10:]:
        sentence = itertools.chain(*[sentence.split(u"。")])
        for x in sentence:
            x = x.strip()
            if x:
                sentences.extend(["%s %s %s" % (sentence_start_token, x, sentence_end_token)])
print len(sentences)
print sentences[0]

4100
START 我坐在会议桌上，跟七个老头一起开会，但会议记录上没有半个字，因为他们在一分钟前全死光了 END


In [221]:
tokenized_sentences = [jieba.lcut(x) for x in sentences]

In [222]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
# find most common words
vocabulary_size = 8000
vocab = word_freq.most_common(vocabulary_size-1)

In [223]:
# build index_to_word and word_to_index vectors
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknow_token)

word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
print "The least frequent word in out vocabulary is '%s' and appeared %d times" % (vocab[-1][0], vocab[-1][1])

# replace all words not in our vocabulary with unknown token
for i, x in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknow_token for w in x]

print "\nExample sentence: '%s'" % sentences[0]
print "\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0]

The least frequent word in out vocabulary is '划开' and appeared 1 times

Example sentence: 'START 我坐在会议桌上，跟七个老头一起开会，但会议记录上没有半个字，因为他们在一分钟前全死光了 END'

Example sentence after Pre-processing: '[u'START', u' ', u'\u6211', u'\u5750\u5728', u'\u4f1a\u8bae', u'\u684c\u4e0a', u'\uff0c', u'\u8ddf', u'\u4e03\u4e2a', u'\u8001\u5934', u'\u4e00\u8d77', u'\u5f00\u4f1a', u'\uff0c', u'\u4f46', u'\u4f1a\u8bae\u8bb0\u5f55', u'\u4e0a', u'\u6ca1\u6709', u'\u534a\u4e2a', u'\u5b57', u'\uff0c', u'\u56e0\u4e3a', u'\u4ed6\u4eec', u'\u5728', u'\u4e00\u5206\u949f', u'\u524d\u5168', u'\u6b7b\u5149', u'\u4e86', u' ', u'END']'


In [224]:
# create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [225]:
x_example, y_example = X_train[10], y_train[10]
print "x:\n%s\n%s" % (" ".join([index_to_word[x] for x in x_example]), x_example)
print "\ny:\n%s\n%s" % (" ".join([index_to_word[y] for y in y_example]), y_example)

x:
START   ” 鬼子 的 声音 从 耳机 传来 ： “ 第八个 董事 的 车刚 到 楼下  
[4, 0, 8, 108, 2, 276, 76, 1973, 2070, 33, 7, 7390, 2673, 2, 6660, 46, 1860, 0]

y:
  ” 鬼子 的 声音 从 耳机 传来 ： “ 第八个 董事 的 车刚 到 楼下   END
[0, 8, 108, 2, 276, 76, 1973, 2070, 33, 7, 7390, 2673, 2, 6660, 46, 1860, 0, 3]


In [226]:
class RNNNumpy:
    
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # random initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))

In [234]:
def forward_propagation(self, x):
    # total number of time steps
    T = len(x)
    s = np.zeros((T+1, self.hidden_dim))
    o = np.zeros((T, self.word_dim))
    for t in np.arange(T):
        s[t] = np.tanh(self.U[:, x[t]] + self.W.dot(s[t-1]))
        o[t] = np.exp(self.V.dot(s[t])) / np.sum(np.exp(self.V.dot(s[t])), axis=0)
    return [o, s]
RNNNumpy.forward_propagation = forward_propagation

In [235]:
def predict(self, x):
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)
RNNNumpy.predict = predict

In [236]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
o, s = model.forward_propagation(X_train[10])
print o.shape
print o

(18, 8000)
[[ 0.00012562  0.00012472  0.00012467 ...,  0.00012588  0.0001254
   0.00012504]
 [ 0.00012374  0.00012466  0.00012592 ...,  0.0001246   0.00012538
   0.00012518]
 [ 0.00012453  0.00012464  0.00012507 ...,  0.00012549  0.00012421
   0.00012485]
 ..., 
 [ 0.00012541  0.00012438  0.00012448 ...,  0.00012558  0.00012533
   0.00012447]
 [ 0.00012484  0.00012549  0.00012448 ...,  0.00012418  0.00012434
   0.00012453]
 [ 0.00012418  0.00012444  0.00012571 ...,  0.00012481  0.00012503
   0.00012471]]


In [241]:
def calculate_total_loss(self, x, y):
    L = 0
    for i in np.arange(len(y)):
        o, s = self.forward_propagation(x[i])
        correct_word_predictions = o[np.arange(len(y[i])), y[i]]
        L += -1 * np.sum(np.log(correct_word_predictions))
    return L

def calculate_loss(self, x, y):
    N = np.sum((len(y_i) for y_i in y))
    return self.calculate_total_loss(x, y) / N

RNNNumpy.calculate_total_loss = calculate_total_loss
RNNNumpy.calculate_loss = calculate_loss

In [242]:
print "Expected Loss for random predictions: %f" % np.log(vocabulary_size)
print "Actual loss: %f" % model.calculate_loss(X_train[:1000], y_train[:1000])

Expected Loss for random predictions: 8.987197
Actual loss: 8.987120


In [243]:
def bptt(self, x, y):
    T = len(y)
    o, s = self.forward_propagation(x)
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            dLdW += np.outer(delta_t, s[bptt_step-1])
            dLdU[:, x[bptt_step]] += delta_t
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

RNNNumpy.bptt = bptt

In [245]:
def numpy_sgd_step(self, x, y, learning_rate):
    dLdU, dLdV, dLdW = self.bptt(x, y)
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

RNNNumpy.sgd_step = numpy_sgd_step

In [246]:
def train_with_sgd(model, X_train, y_train, learning_rate=0.005, nepoch=100, evaluate_loss_after=5):
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
            
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5
                print "setting learning_rate to %f" % learning_rate
                sys.stdout.flush()
        for i in range(len(y_train)):
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [256]:
np.random.seed(10)
model = RNNNumpy(vocabulary_size)
losses = train_with_sgd(model, X_train[:1000], y_train[:1000], nepoch=10, evaluate_loss_after=1)

2017-08-14 09:04:40: Loss after num_examples_seen=0 epoch=0: 8.987120
2017-08-14 09:07:28: Loss after num_examples_seen=1000 epoch=1: 5.926527
2017-08-14 09:10:09: Loss after num_examples_seen=2000 epoch=2: 5.795725
2017-08-14 09:12:40: Loss after num_examples_seen=3000 epoch=3: 5.733097
2017-08-14 09:15:26: Loss after num_examples_seen=4000 epoch=4: 5.686774
2017-08-14 09:18:12: Loss after num_examples_seen=5000 epoch=5: 5.639580
2017-08-14 09:20:49: Loss after num_examples_seen=6000 epoch=6: 5.609171
2017-08-14 09:23:28: Loss after num_examples_seen=7000 epoch=7: 5.587945
2017-08-14 09:26:26: Loss after num_examples_seen=8000 epoch=8: 5.576130
2017-08-14 09:29:07: Loss after num_examples_seen=9000 epoch=9: 5.566501


In [252]:
def generate_sentence(model):
    new_sentence = [word_to_index[sentence_start_token]]
    while not new_sentence[-1] == word_to_index[sentence_end_token]:
        next_word_probs, _ = model.forward_propagation(new_sentence)
        sampled_word = word_to_index[unknow_token]
        # we don't want to sample unknown words
        while sampled_word == word_to_index[unknow_token]:
            samples = np.random.multinomial(1, next_word_probs[-1])
            sampled_word = np.argmax(samples)
        new_sentence.append(sampled_word)
    sentence_str = [index_to_word[x] for x in new_sentence[1:-1]]
    return sentence_str

In [262]:
num_sentence = 10
sentence_min_length = 10

for i in range(num_sentence):
    sent = []
    # we want long sentences, not sentences with one or two words
    while len(sent) < sentence_min_length:
        sent = generate_sentence(model)
    print " ".join(sent)
    print "-"*20

  一行 猜 炫耀 事情 远 在 推说 ， 脸上 口袋 差不多 跑 见过 的 哭 ， 我 不是 的 最好 子弹 很 一个 更  
--------------------
  Chapter 归 约定 掉泪 摆烂 了 身上 的 假装 几句 ， 走光 他 的 爆炸 悲剧 时机 的 要 离开 之间 了 呆呆 的 但是 又 今天 手法 小熹 腰际 石灰粉  
--------------------
  “ 小熹 职业杀手 的 鸡 空气 ， 每 有 走 的 子弹 沿路 里 命 有疤 的 感觉 桌旁 麦克风 所 小熹 自己 里 了  
--------------------
  就 我 的 练习 上帝 了 快 的 时候 状况 我 那些 出现 低头 投胎 ， 抢匪 教堂 的 学校 ， 他 手法 ， 比 在 活着 小熹 ， 凝视着 手 还 点头 简称 他 姊 预备 都 用 堆 口 我 的 咬 ？ 神不知鬼不觉 香草 差不多 的 独有 干 仇好 安稳 ： ， 爆炸 差不多 神父 东西 ， 而是 一张 威尼斯 一大堆 他 来 ， 如果 就 杂碎  
--------------------
  那 的 以前 怎么 的 职业杀手 ， 我 整个 不是 的 想起  
--------------------
  于是 ！ … 其实 的 忍不住 ， 眼睛 将 ， 然后 无话可说 白头发 照明 都 在 过 的 脱掉 里  
--------------------
  的 一阵子 知道 ， 当地 的 连结 扯 额外 跑 柜台 了 任务 多 着 方向 的 牛奶 ， 他 忽然 逮到 嘶吼 他 的 式 ， 抬起 上 飞机 职业杀手 瞬间 跑 人 就是 乐此不疲 ， 给 拯救 的 电视 营养 ， 公司 另 口味  
--------------------
  在 都 按照 黏 命 激滚 在 酝酿 什么 拆解 喉咙 十一 没 、 一段时间 ， 了 插 一个 扔 ！ 严肃 我 的 间谍 尸体 握住 男用 的 价值 了 搞 做爱 首尔 谈恋爱 要 一种 乖孩子 热水 我 后来 等  
--------------------
  我 脱北者 啦 ， 我 想念 肚子 的 手法 里 内 ， 现在 拼图 ， 等到 有 分手 又 其中 的 开始 ， 然后 我 ， 我 赤脚 每次  
-------------